    %reload_ext XXX.monkey
    ---
    %%monkey file -- args
    trace

In [1]:
from monkeytype.tracing import CallTraceLogger, trace_calls
from monkeytype.stubs import build_module_stubs_from_traces
DUNDER = '__%s__'

In [2]:
class IPythonTraceLogger(CallTraceLogger):
    """A CallTraceLogger that stores logged traces in a CallTraceStore."""
    def __init__(self, data=None) -> None:
        self.data = None or []
        self.traces = []

    def log(self, trace) -> None:
        self.traces.append(trace)

    def flush(self) -> None:
        self.data.extend(self.traces)
        self.traces = []
        
    def stubs(logger): return build_module_stubs_from_traces(logger.data).get(DUNDER%'main').render()

In [27]:
from contextlib import contextmanager

In [28]:
@contextmanager
def trace(logger=None):
    logger = logger or IPythonTraceLogger()
    with trace_calls(logger):
        yield logger
        return None

In [29]:
def monkey(line, cell):
    ip=__import__('IPython').get_ipython() 
    with trace() as logger:  
        eval(cell, ip.user_ns, ip.user_ns)
       
    if line: ip.magics_manager.magics['cell']['file'](line, logger.stubs() + '\n')
    else:  ip.set_next_input(logger.stubs())

In [30]:
def load_ipython_extension(ip=__import__('IPython').get_ipython()):
    ip.register_magic_function(monkey, 'cell')